In [ ]:
!pip install git+https://github.com/PennyLaneAI/pennylane/@master
!pip install git+https://github.com/QSAR-UBC/ionizer@main
!pip install qiskit

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import Identity, PauliX, PauliY, PauliZ

import ionizer
from ionizer.transforms import ionize, single_qubit_fusion_gpi, commute_through_ms_gates

In [2]:
coeffs = np.array([
    -10.943495,   6.590075,   6.59009 ,   6.59008 ,   6.1535  ,
    -3.991875,   3.991875,  -0.56607 ,  -0.400535,  -0.334915,
    -0.841875,   0.34829 ,  -0.665395,  -0.33492 ,  -0.545725,
     0.644445,  -0.334915,  -0.39765 ,   0.792515,  -0.282125,
     0.282125,   0.225   ,   0.34782 ,   0.34782 ,  -0.182285,
    -0.182285,  -0.419445,  -0.419445,  -0.667545,  -0.667545,
     0.24693 ,   0.24693 ,   0.44714 ,   0.44714 ,   0.419445,
     0.419445,   0.415175,   0.415175,  -0.4993  ,  -0.4993  ,
    -0.419445,  -0.419445,  -0.28899 ,  -0.28899 ,   0.625485,
     0.625485,  -0.22309 ,  -0.22309 ,   0.22309 ,   0.22309 ,
     0.27825 ,   0.27825 
])

ops = [
    Identity(wires=[0]),
    PauliZ(wires=[0]),
    PauliZ(wires=[1]),
    PauliZ(wires=[2]),
    PauliZ(wires=[3]),
    PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]),
    PauliZ(wires=[0]) @ PauliZ(wires=[2]),
    PauliZ(wires=[0]) @ PauliZ(wires=[3]),
    PauliZ(wires=[0]) @ PauliZ(wires=[4]),
    PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[1]) @ PauliZ(wires=[2]),
    PauliZ(wires=[1]) @ PauliZ(wires=[3]),
    PauliZ(wires=[1]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[2]) @ PauliZ(wires=[3]),
    PauliZ(wires=[2]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]),
    PauliY(wires=[0]) @ PauliY(wires=[1]),
    PauliX(wires=[0]) @ PauliX(wires=[1]),
    PauliY(wires=[0]) @ PauliY(wires=[2]),
    PauliX(wires=[0]) @ PauliX(wires=[2]),
    PauliY(wires=[0]) @ PauliY(wires=[3]),
    PauliX(wires=[0]) @ PauliX(wires=[3]),
    PauliY(wires=[0]) @ PauliY(wires=[4]),
    PauliX(wires=[0]) @ PauliX(wires=[4]),
    PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[0]),
    PauliY(wires=[1]) @ PauliY(wires=[2]),
    PauliX(wires=[1]) @ PauliX(wires=[2]),
    PauliY(wires=[1]) @ PauliY(wires=[3]),
    PauliX(wires=[1]) @ PauliX(wires=[3]),
    PauliY(wires=[1]) @ PauliY(wires=[4]),
    PauliX(wires=[1]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliX(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[1]),
    PauliY(wires=[2]) @ PauliY(wires=[3]),
    PauliX(wires=[2]) @ PauliX(wires=[3]),
    PauliY(wires=[2]) @ PauliY(wires=[4]),
    PauliX(wires=[2]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliX(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[2]),
    PauliY(wires=[3]) @ PauliY(wires=[4]),
    PauliX(wires=[3]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliX(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[3]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliX(wires=[4]),
    PauliX(wires=[4])
]

In [3]:
H_tapered = qml.Hamiltonian(coeffs, ops)

In [4]:
dev = qml.device("default.qubit", wires=range(len(H_tapered.wires)))

@qml.qnode(dev)
def tapered_circuit_simplified(params):
    qml.RY(params[0], wires=dev.wires[0])
    for idx in range(len(dev.wires)-1):
        qml.SingleExcitation.compute_decomposition(params[idx+1], wires=[dev.wires[idx], dev.wires[idx+1]])
    return qml.expval(H_tapered)

In [5]:
params = np.array([ 3.26996631,  2.26064568,  2.15960289, -2.01268257, -0.19100396], requires_grad=True)

## Convert to trapped ion gates

In [6]:
@qml.qnode(dev)
@single_qubit_fusion_gpi
@commute_through_ms_gates(direction="right")
@ionize
def tapered_circuit_ionized(params):
    qml.RY(params[0], wires=dev.wires[0])
    for idx in range(len(dev.wires)-1):
        qml.SingleExcitation.compute_decomposition(params[idx+1], wires=[dev.wires[idx], dev.wires[idx+1]])
    return qml.expval(H_tapered)
                      
print(np.round(tapered_circuit_ionized(params), decimals=6))

-10.853163


In [7]:
print(qml.draw(tapered_circuit_ionized)(params))

0: ──GPI2(3.14)───GPI(2.81)───GPI2(2.36)─╭MS──GPI(0.57)───────────────────────────╭MS──GPI2(-1.09)
1: ──GPI2(-2.36)──GPI(-2.75)──GPI2(1.57)─╰MS──GPI2(-1.57)──GPI(1.79)──GPI2(-2.70)─╰MS─╭MS─────────
2: ──GPI2(-2.36)──GPI(-2.75)──GPI2(1.57)──────────────────────────────────────────────╰MS─────────
3: ──GPI2(-2.36)──GPI(-2.75)──GPI2(1.57)──────────────────────────────────────────────────────────
4: ──GPI2(-2.36)──GPI(-2.75)──GPI2(1.57)──────────────────────────────────────────────────────────

───GPI(-1.96)───GPI2(0.30)────────────────────────────────────────────────────────────────────────
───GPI(0.54)────────────────────────────╭MS──GPI2(-1.09)──GPI(-1.96)───GPI2(0.30)─────────────────
───GPI2(-1.57)──GPI(-1.33)──GPI2(-2.65)─╰MS─╭MS───────────GPI(-0.50)───────────────────────────╭MS
────────────────────────────────────────────╰MS───────────GPI2(-1.57)──GPI(-0.28)──GPI2(-0.56)─╰MS
──────────────────────────────────────────────────────────────────────────────────────────────────

────────

In [8]:
qml.specs(tapered_circuit_ionized)(params)

/home/jovyan/.local/lib/python3.9/site-packages/pennylane/tape/qscript.py:1386: UserWarning: The gate_sizes key is deprecated and will be removed in the next release. Going forward, please use: qml.specs()["resources"].gate_sizes
  warnings.warn(
/home/jovyan/.local/lib/python3.9/site-packages/pennylane/tape/qscript.py:1386: UserWarning: The gate_types key is deprecated and will be removed in the next release. Going forward, please use: qml.specs()["resources"].gate_types
  warnings.warn(


{'resources': Resources(num_wires=5, num_gates=54, gate_types=defaultdict(<class 'int'>, {'GPI2': 28, 'GPI': 18, 'MS': 8}), gate_sizes=defaultdict(<class 'int'>, {1: 46, 2: 8}), depth=26, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 46, 2: 8}),
 'gate_types': defaultdict(int, {'GPI2': 28, 'GPI': 18, 'MS': 8}),
 'num_operations': 54,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 56,
 'depth': 26,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Split into commuting sets

In [8]:
H_tapered_grouped = qml.Hamiltonian(H_tapered.coeffs, H_tapered.ops, grouping_type="qwc")
ham_tapes, ham_fn = qml.transforms.hamiltonian_expand(tapered_circuit_ionized.qtape, group=True)

In [9]:
ham_tapes = [ionize.tape_fn(tape) for tape in ham_tapes]

In [10]:
res = qml.execute(ham_tapes, dev)

In [11]:
ham_fn(res)

array(-10.85316266)

In [12]:
from azure.quantum.cirq import AzureQuantumService
from azure.quantum import Workspace

service = AzureQuantumService(
   resource_id = "",
   location = "westus2"
)

workspace = Workspace(
   resource_id = "",
   location = "westus2"
)

In [25]:
target, target_string = workspace.get_targets(name="ionq.qpu.aria-1"), "HW"
# target, target_string = workspace.get_targets(name="ionq.simulator"), "SIM"

In [26]:
shots = 1000
isotope = 18

job_circuits = [ionizer.utils.tape_to_json(tape, f"HW-O{isotope}-{target_string}-{idx}", shots=shots) for idx, tape in enumerate(ham_tapes)]

In [27]:
for idx, circuit in enumerate(job_circuits):
    cost = target.estimate_cost(circuit["body"], num_shots=shots)
    print(f"Estimated cost of circuit {idx}: {cost.estimated_total} ")

Estimated cost of circuit 0: 11.907 
Estimated cost of circuit 1: 11.907 
Estimated cost of circuit 2: 11.907 
Estimated cost of circuit 3: 11.907 
Estimated cost of circuit 4: 11.907 
Estimated cost of circuit 5: 11.907 
Estimated cost of circuit 6: 11.907 
Estimated cost of circuit 7: 11.907 


In [ ]:
jobs = []

for idx, circuit in enumerate(job_circuits):
    jobs.append(target.submit(circuit["body"], name=f"O{isotope}-{target_string}-{idx}", num_shots=shots))
    print(jobs[-1].id)

In [13]:
job_ids = [
    ""
]

jobs = [workspace.get_job(id) for id in job_ids]

In [14]:
job_results = []
for job in jobs:
    job_results.append(job.get_results())

In [31]:
def eigenvalue(pauli, bitstring):
    """Given a Pauli word and a bitstring, return whether it is a +1 or -1 
    eigenvalue."""
    eigval = 1
    
    pauli_string = qml.pauli.pauli_word_to_string(pauli, wire_map={i: i for i in range(len(dev.wires))})

    for bit, p in zip(bitstring, pauli_string):
        if bit == "1" and p != "I":
            eigval *= -1
    
    return eigval

In [32]:
def hist_to_counts(hist, num_shots, reverse=True):
    if reverse:
        return {np.binary_repr(int(key), 5)[::-1]: int(np.round(num_shots * val)) for key, val in list(hist.items())}
    return {np.binary_repr(int(key), 5): int(np.round(num_shots * val)) for key, val in list(hist.items())}


In [33]:
sim_results = [hist_to_counts(result['histogram'], num_shots=shots) for result in job_results]

In [34]:
processed_results = []

for idx, results in enumerate(sim_results):
    results_this_circuit = []
    
    # Get the Hamiltonian terms measured in this case
    ham_terms = [H_tapered_grouped.ops[op_idx] for op_idx in H_tapered_grouped.grouping_indices[idx]]
    
    for term in list(ham_terms):
        expval = 0.0
        
        for bitstring, count in list(results.items()):
            sign = eigenvalue(term, bitstring)
            expval += sign * count
        results_this_circuit.append(expval / shots)
            
    processed_results.append(np.array(results_this_circuit))
    
processed_results = np.array(processed_results)

/home/jovyan/.local/lib/python3.9/site-packages/autograd/numpy/numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return _np.array(args, *array_args, **array_kwargs)


In [35]:
processed_results

tensor([tensor([ 1.   ,  0.55 ,  0.606,  0.104,  0.968, -0.612,  0.176, -0.274,
                 0.542, -0.222,  0.602,  0.108,  0.136, -0.116], requires_grad=True),
        tensor([ 0.65 ,  0.286, -0.166,  0.616, -0.5  , -0.014, -0.006], requires_grad=True),
        tensor([ 0.004,  0.234, -0.586, -0.022], requires_grad=True),
        tensor([-0.906, -0.05 , -0.888], requires_grad=True),
        tensor([ 0.02 , -0.036], requires_grad=True),
        tensor([-0.06 ,  0.038], requires_grad=True),
        tensor([-0.34 ,  0.35 ,  0.46 ,  0.042, -0.374, -0.512, -0.058,  0.558,
                 0.052,  0.082], requires_grad=True)                           ,
        tensor([-0.398, -0.582, -0.068,  0.524,  0.07 ,  0.118, -0.398,  0.328,
                 0.496,  0.026], requires_grad=True)                           ],
       dtype=object, requires_grad=True)

In [36]:
ham_fn(processed_results)

tensor(-10.54573377, requires_grad=True)